In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')
sys.path.insert(0, '../driftpy/src/')

import pandas as pd 
import numpy as np 

pd.options.plotting.backend = "plotly"

from driftpy.constants.config import configs
from anchorpy import Provider, Wallet
from solana.keypair import Keypair
from solana.rpc.async_api import AsyncClient
from driftpy.clearing_house import ClearingHouse
from driftpy.accounts import *
from driftpy.constants.numeric_constants import * 

In [ ]:
def human_amm_df(amm: AMM):
    bool_fields = [ 'last_oracle_valid']
    enum_fields = ['oracle_source']
    pure_fields = ['last_update_slot', 'long_intensity_count', 'short_intensity_count', 
    'curve_update_intensity', 'amm_jit_intensity'
    ]
    reserve_fields = [
        'base_asset_reserve', 'quote_asset_reserve', 'min_base_asset_reserve', 'max_base_asset_reserve', 'sqrt_k'
        'ask_base_asset_reserve', 'ask_quote_asset_reserve', 'bid_base_asset_reserve', 'bid_quote_asset_reserve'
        'terminal_quote_asset_reserve', 'base_asset_amount_long', 'base_asset_amount_short', 'base_asset_amount_with_amm', 'base_asset_amount_with_unsettled_lp',
        'user_lp_shares'
        ]
    pct_fields = ['long_spread', 'short_spread', 'concentration_coef',]
    funding_fields = ['cumulative_funding_rate_long', 'cumulative_funding_rate_short', 'last_funding_rate', 'last_funding_rate_long', 'last_funding_rate_short', 'last24h_avg_funding_rate']
    quote_asset_fields = ['total_fee', 'total_mm_fee', 'total_exchange_fee', 'total_fee_minus_distributions',
    'total_fee_withdrawn', 'total_liquidation_fee', 'cumulative_social_loss', 'net_revenue_since_last_funding',
    'quote_asset_amount_long', 'quote_asset_amount_short', 'quote_entry_amount_long', 'quote_entry_amount_short',
    'volume24h', 'long_intensity_volume', 'short_intensity_volume',]
    time_fields = ['last_mark_price_twap_ts', 'last_oracle_price_twap_ts']
    duration_fields = ['lp_cooldown_time']
    px_fields = ['last_bid_price_twap', 'last_ask_price_twap', 'last_mark_price_twap', 'last_mark_price_twap5min',
    'peg_multiplier',
    'mark_std']
    pool_fields = ['fee_pool']


def human_market_df(market: PerpMarket):
    enum_fields = ['status', 'contract_tier', '']
    pure_fields = ['number_of_users', 'market_index', 'next_curve_record_id', 'next_fill_record_id', 'next_funding_rate_record_id']
    pct_fields = ['imf_factor', 'unrealized_pnl_imf_factor', 'liquidator_fee', 'if_liquidation_fee',
    'margin_ratio_initial', 'margin_ratio_maintenance']
    px_fields = ['expiry_price', ]
    time_fields = ['last_trade_ts', 'expiry_ts']
    pool_fields = ['pnl_pool']
    
    
    

In [ ]:
config = configs['devnet']
import os

url = 'https://api.devnet.solana.com'
import json
print(config)
# random key 
with open(os.path.expanduser('/Users/zanes/.config/solana/id.json'), 'r') as f: secret = json.load(f) 
kp = Keypair.from_secret_key(bytes(secret))
# print('pk:', kp.public_key)
wallet = Wallet(kp)
connection = AsyncClient(url)
provider = Provider(connection, wallet)
ch = ClearingHouse.from_config(config, provider)

In [ ]:
state = await get_state_account(ch.program)
state

In [ ]:
market = await get_perp_market_account(ch.program, 0)